In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from PyPDF3 import PdfFileReader
import io
from chemdataextractor.doc import Document, Title, Paragraph
from chemdataextractor.scrape import Selector
from chemdataextractor.scrape.pub.rsc import RscHtmlDocument
from tqdm.notebook import tqdm
import time

In [2]:
keyword = 'COVID-19'
# keyword = 'SARS-CoV-2'

In [3]:
max_papers = 50

In [4]:
nature_search = 'https://www.nature.com/search?q='+keyword+'&journal='
nature = 'https://www.nature.com/'
pages = '&page='

In [5]:
def scrapping(url): 
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        text_list = html.split('\n')
    else : 
        print(response.status_code)
        text_list = None
    return text_list

In [7]:
start_page = 1

In [8]:
keyword = '%20'.join(keyword.split(' '))

df = pd.DataFrame(columns=['Articles', 'DOI', 'Journal', 'Authors', 'Published', 'Target'])

text_list = scrapping(nature_search)
pg = 0
for i in range(len(text_list)):
    if 'inline-group-item inline-group-middle" data-test="' in text_list[i]:
        page_range = text_list[i].split('" data-page="')[1].split('">')[0]
        pg = 1
    if pg == 0:
        page_range = 1

print(page_range)

num_of_paper = 0
for p in tqdm(range(start_page, int(page_range)+1)):
    if p == 1:
        search = nature_search
    else:
        search = nature_search+pages+str(p)
    
    text_list = scrapping(search)
    for i in range(len(text_list)):
        if '<a href="/articles/' in text_list[i]:
            url = text_list[i].split('<a href="/')[1].split('" itemprop="')[0]
            articles = text_list[i+1].strip()
            articles = articles.replace('<sub>', '').replace('</sub>', '').replace('<i>', '').replace('</i>', '')
            page_list = scrapping(nature+url)
            for j in range(len(page_list)):
                if '"doi":"' in page_list[j]:
                    doi = 'https://doi.org/'+page_list[j].split('"doi":"')[1].split('"},"')[0]
                    response = requests.get(doi)
                    html = response.text
                    soup = BeautifulSoup(html, 'html.parser')
                    if 'Download PDF' in soup.get_text():
                        try:
                            sel = Selector.from_text(html)
                            scrape = RscHtmlDocument(sel)

                            f = open("./test.html", 'w', encoding='UTF-8')
                            f.write(html.replace('Download PDF', ''))
                            f.close()
                            f = open('./test.html', 'rb')
                            doc = Document.from_file(f)
                            f.close()

                            paper_list = html.split('\n')
                            section_list = []
                            for x in range(len(paper_list)-1):
                                if 'section data-title="' in paper_list[x]:
                                    part = paper_list[x].split('</section>')
                                    for p in part:
                                        if 'section data-title="' in p:
                                            section = p.split('section data-title="')[1].split('">')[0]
                                            section_list.append(section)
                            if 'Abstract' in [doc.elements[t].text for t in range(len(doc.elements)) 
                                              if str(type(doc.elements[t])) == "<class 'chemdataextractor.doc.text.Heading'>"]:
                                section_list = ['Abstract'] + section_list

                            start = 0
                            end = 0
                            data_list = []
                            for n in range(len(doc.elements)):
                                if str(type(doc.elements[n])) == "<class 'chemdataextractor.doc.text.Heading'>":
                                    if start == 0: 
                                        if doc.elements[n].text in section_list:
                                            start = 1
                                    if doc.elements[n].text == 'References':
                                        break
                                if start == 1:
                                    if str(type(doc.elements[n])) == "<class 'chemdataextractor.doc.text.Paragraph'>":
                                        data_list.append(doc.elements[n].text)
                                    if str(type(doc.elements[n])) == "<class 'chemdataextractor.doc.figure.Figure'>":
                                        data_list.append(doc.elements[n].caption.text)

                                        
                            full_text = []
                            for text in data_list:
                                full_text.append(text)

                            target = full_text
                            df = df.append({'Articles':articles, 
                                            'DOI':scrape.doi, 
                                            'Journal':scrape.publisher, 
                                            'Authors':scrape.authors, 
                                            'Published':scrape.published_date, 
                                            'Target':target}, ignore_index=True)
                            
                            print(num_of_paper+1, doi)
                            num_of_paper += 1
                            if num_of_paper == max_papers:
                                break

                        except:
                            print('Error :', doi)
                            articles, doi, journal, authors, published, target = None, None, None, None, None, None
                            continue
                            
        if num_of_paper == max_papers:
            break
            
#     df.to_json(f'Nature_{num_of_paper+1}.json', orient='table')
#     df.to_json(f'Nature_{start_page}.json', orient='table')
    
    if num_of_paper == max_papers:
        break

124


  0%|          | 0/124 [00:00<?, ?it/s]

1 https://doi.org/10.1038/s41586-021-03569-1
2 https://doi.org/10.1038/s41586-021-03530-2
3 https://doi.org/10.1038/s41586-021-03553-9
4 https://doi.org/10.1038/s41586-021-03570-8
5 https://doi.org/10.1038/s41591-021-01366-x
6 https://doi.org/10.1038/s41562-021-01112-w
7 https://doi.org/10.1038/s41591-021-01355-0
8 https://doi.org/10.1038/s41467-021-22214-z
9 https://doi.org/10.1038/s41598-021-88679-6
10 https://doi.org/10.1038/s41467-021-22474-9
11 https://doi.org/10.1038/s41591-021-01337-2
12 https://doi.org/10.1038/s41467-021-22713-z
13 https://doi.org/10.1038/s41467-021-22601-6
14 https://doi.org/10.1038/s41588-021-00854-7
15 https://doi.org/10.1038/s41591-021-01360-3
16 https://doi.org/10.1038/s41467-021-22423-6
17 https://doi.org/10.1038/s41591-021-01329-2
18 https://doi.org/10.1038/s41591-021-01310-z
19 https://doi.org/10.1038/s41551-021-00704-1
20 https://doi.org/10.1038/s41467-021-22446-z
21 https://doi.org/10.1038/s41582-021-00502-y
22 https://doi.org/10.1038/s41467-021-22366

In [9]:
df

,Articles,DOI,Journal,Authors,Published,Target
0,A molecular single-cell lung atlas of lethal C...,doi:10.1038/s41586-021-03569-1,Nature Publishing Group,"[Johannes C. Melms, Jana Biermann, Huachao Hua...",2021-04-29,[Respiratory failure is the leading cause of d...
1,Adjuvanting a subunit COVID-19 vaccine to indu...,doi:10.1038/s41586-021-03530-2,Nature Publishing Group,"[Prabhu S. Arunachalam, Alexandra C. Walls, Na...",2021-04-19,[The development of a portfolio of COVID-19 va...
2,High-dimensional characterization of post-acut...,doi:10.1038/s41586-021-03553-9,Nature Publishing Group,"[Ziyad Al-Aly, Yan Xie, Benjamin Bowe, Ziyad A...",2021-04-22,[The acute clinical manifestations of COVID-19...
3,COVID-19 tissue atlases reveal SARS-CoV-2 path...,doi:10.1038/s41586-021-03570-8,Nature Publishing Group,"[Toni M. Delorey, Carly G. K. Ziegler, Graham ...",2021-04-29,"[COVID-19, caused by SARS-CoV-2, can result in..."
4,Author Correction: What do journalists say abo...,doi:10.1038/s41591-021-01366-x,Nature Publishing Group,"[Anita Makri, Anita Makri]",2021-05-04,"[Reprints and Permissions, Makri, A. Author Co..."
5,Anti-intellectualism and the mass public’s res...,doi:10.1038/s41562-021-01112-w,Nature Publishing Group,"[Eric Merkley, Peter John Loewen, Eric Merkley...",2021-04-28,[Anti-intellectualism (the generalized distrus...
6,Delayed production of neutralizing antibodies ...,doi:10.1038/s41591-021-01355-0,Nature Publishing Group,"[Carolina Lucas, Jon Klein, Maria E. Sundaram,...",2021-05-05,[Recent studies have provided insights into in...
7,Hospital load and increased COVID-19 related m...,doi:10.1038/s41467-021-22214-z,Nature Publishing Group,"[Hagai Rossman, Tomer Meir, Jonathan Somer, Sm...",2021-03-26,[The spread of Coronavirus disease 19 (COVID-1...
8,The COVID-19 lab score: an accurate dynamic to...,doi:10.1038/s41598-021-88679-6,Nature Publishing Group,"[Pablo Jose Antunez Muiños, Diego López Otero,...",2021-04-30,[Deterioration is sometimes unexpected in SARS...
9,Leveraging community mortality indicators to i...,doi:10.1038/s41467-021-22474-9,Nature Publishing Group,"[Oliver J. Watson, Mervat Alhaffar, Zaki Mehch...",2021-04-22,[The COVID-19 pandemic has resulted in substan...


In [10]:
# df.to_json('Nature_Total.json', orient='table') # SARS-CoV-2도 진행하여 합한다.